# 5 Equity Index Long Term Reversal strategy (REV)


(a) 
Construct the return to a long-short reversal strategy portfolio. To that effect every month sort currency hedged stock indexes based on their 12-month lagged 5-year past return (that is in month t rank stocks based on their t−60 to t−12 cumulative return). Call Rankit the corresponding rank of index i at time t.
Then compute monthly returns to a portfolio that invests in index i the weight: wit = Z ((N + 1)/2 − Ranki) for all i = 1, . . . , N, and where N is the total number of stock indexes traded and Z is a factor that insures that the the sum of the long positions is +$1 and the sum of the short positons is −$1.

In [2]:
import pandas as pd
import numpy as np
from scipy import stats


In [4]:
# Load the dataset and set 'date' as the index
df = pd.read_csv("merged_all_data.csv", parse_dates=['date'])
df.set_index('date', inplace=True)

# Define the list of countries and prefix for currency-hedged returns
countries = ['AUSTRALIA', 'SWITZERLAND', 'GERMANY', 'FRANCE', 'UNITED KINGDOM', 'JAPAN']
prefix = 'mportretx_'  # monthly portfolio returns

# Create a DataFrame for long-term reversal signals
rev_signal = pd.DataFrame(index=df.index)

# Compute past t−60 to t−12 cumulative log-returns
for country in countries:
    col = prefix + country
    log_ret = np.log(1 + df[col])
    rev_signal[country] = log_ret.rolling(window=49, min_periods=49).sum().shift(12)  # t−60 to t−12 returns

# Rank countries each month based on reversal signal
rev_ranks = rev_signal.rank(axis=1, method='first')

# Compute ranking weights for the long-short reversal portfolio
N = len(countries)
Z = 8/N/N
rev_weights = ((N + 1)/2 - rev_ranks) * Z  # reversal: long past losers, short past winners
print(rev_weights)
# Extract returns8^1
returns = df[[prefix + c for c in countries]].copy()
returns.columns = countries

# Compute REV strategy return
REV_return = (rev_weights * returns).sum(axis=1)

REV_return[rev_weights.isnull().any(axis=1)] = np.nan


            AUSTRALIA  SWITZERLAND   GERMANY    FRANCE  UNITED KINGDOM  \
date                                                                     
2002-01-31        NaN          NaN       NaN       NaN             NaN   
2002-02-28        NaN          NaN       NaN       NaN             NaN   
2002-03-28        NaN          NaN       NaN       NaN             NaN   
2002-04-30        NaN          NaN       NaN       NaN             NaN   
2002-05-31        NaN          NaN       NaN       NaN             NaN   
...               ...          ...       ...       ...             ...   
2024-08-30   0.333333     0.111111 -0.111111 -0.333333        0.555556   
2024-09-30   0.333333     0.111111 -0.111111 -0.333333        0.555556   
2024-10-31   0.111111     0.333333 -0.111111 -0.333333        0.555556   
2024-11-29   0.111111     0.333333 -0.111111 -0.333333        0.555556   
2024-12-31  -0.111111     0.333333  0.111111 -0.333333        0.555556   

               JAPAN  
date          

In [5]:
print(REV_return.head(13))

date
2002-01-31   NaN
2002-02-28   NaN
2002-03-28   NaN
2002-04-30   NaN
2002-05-31   NaN
2002-06-28   NaN
2002-07-31   NaN
2002-08-30   NaN
2002-09-30   NaN
2002-10-31   NaN
2002-11-29   NaN
2002-12-31   NaN
2003-01-31   NaN
dtype: float64


(b) Compute and compare the mean, standard deviation, and Sharpe ratios of the long
and short legs of the strategy as well as of the strategy itself. Test if the strategy has
an average return that is statistically significantly different from zero.

In [6]:
# Define long and short positions based on ranks
long_mask_rev = rev_ranks.apply(lambda row: row <= 3, axis=1)
short_mask_rev = rev_ranks.apply(lambda row: row >= (N -3 + 1), axis=1)

# Compute average return of long and short legs
long_returns_rev = (returns * long_mask_rev).sum(axis=1) / long_mask_rev.sum(axis=1)
short_returns_rev = (returns * short_mask_rev).sum(axis=1) / short_mask_rev.sum(axis=1)

# Summary statistics
rev_results = pd.DataFrame({
    "Mean": [long_returns_rev.mean(), short_returns_rev.mean(), REV_return.mean()],
    "Std": [long_returns_rev.std(), short_returns_rev.std(), REV_return.std()],
    "Sharpe": [
        long_returns_rev.mean() / long_returns_rev.std(),
        short_returns_rev.mean() / short_returns_rev.std(),
        REV_return.mean() / REV_return.std()
    ]
}, index=["Long", "Short", "Strategy"])

# T-test for significance
t_stat, p_value = stats.ttest_1samp(REV_return.dropna(), popmean=0)
rev_results["t-stat"] = [np.nan, np.nan, t_stat]
rev_results["p-value"] = [np.nan, np.nan, p_value]

# Display result rounded to 4 decimal places
print(rev_results.round(4))

            Mean     Std  Sharpe  t-stat  p-value
Long      0.0023  0.0393  0.0579     NaN      NaN
Short     0.0028  0.0398  0.0693     NaN      NaN
Strategy -0.0007  0.0229 -0.0311 -0.4574   0.6478


(c) Regress the REV strategy return on the DIV return. Interpret the regression results.
In particular, do you think that it is interesting for a DIV-investor to also invest in the
REV strategy?

In [7]:
import statsmodels.api as sm
import pandas as pd

# Load DIV strategy returns
DIV_return = pd.read_csv("returns_rp_hedged.csv", parse_dates=['date'])
DIV_return.set_index('date', inplace=True)

# Align common time frame 
returns_df = pd.concat([REV_return, DIV_return], axis=1, join='inner')
returns_df.columns = ['REV', 'DIV']

# Delete missing values
returns_df = returns_df.dropna()

# Regression
X = sm.add_constant(returns_df['DIV'])  # add intercept
y = returns_df['REV']
model = sm.OLS(y, X).fit()

# print
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    REV   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.875
Date:                Wed, 04 Jun 2025   Prob (F-statistic):              0.172
Time:                        14:20:26   Log-Likelihood:                 503.52
No. Observations:                 213   AIC:                            -1003.
Df Residuals:                     211   BIC:                            -996.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -7.288e-05      0.002     -0.046      0.9